<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Attention_Mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(input_dim, hidden_dim)
        self.context = nn.Parameter(torch.randn(hidden_dim, 1))

    def forward(self, x):
        # Calculate attention scores
        attn_scores = torch.matmul(F.tanh(self.attention(x)), self.context)
        attn_scores = F.softmax(attn_scores, dim=1)
        weighted_sum = torch.sum(attn_scores * x, dim=1)
        return weighted_sum

input_data = torch.randn(32, 10, 64)  # Batch of 32, 10 time steps, 64 features
attention_layer = Attention(input_dim=64, hidden_dim=32)
output = attention_layer(input_data)
print(output.shape)